In [34]:
from sklearn.feature_extraction.text import HashingVectorizer, VectorizerMixin

In [2]:
??HashingVectorizer

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
docs = ['this this this book',
        'this cat good',
        'cat good shit']
count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model
X = count_model.fit_transform(docs)
#X[X > 0] = 1 # run this line if you don't want extra within-text cooccurence (see below)
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0
print(Xc.todense()) # print out matrix in dense format

[[0 0 0 0 3]
 [0 0 2 1 1]
 [0 2 0 1 1]
 [0 1 1 0 0]
 [3 1 1 0 0]]


In [7]:
count_model.vocabulary_

{'this': 4, 'book': 0, 'cat': 1, 'good': 2, 'shit': 3}

In [19]:
count_model.get_feature_names()

['book', 'cat', 'good', 'shit', 'this']

In [10]:
X.todense()

matrix([[1, 0, 0, 0, 3],
        [0, 1, 1, 0, 1],
        [0, 1, 1, 1, 0]], dtype=int64)

In [11]:
docs

['this this this book', 'this cat good', 'cat good shit']

In [12]:
docs = [s.split() for s in docs]

In [13]:
docs

[['this', 'this', 'this', 'book'],
 ['this', 'cat', 'good'],
 ['cat', 'good', 'shit']]

In [16]:
cm = CountVectorizer(ngram_range=(1,1))

In [17]:
cm.fit_transform(docs)

AttributeError: 'list' object has no attribute 'lower'

In [129]:
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer


class Cooccurrence(CountVectorizer):
    """Word-word Co-ocurrence matrix
    Convert collection of sentences to word-word co-ocurrence matrix
    
    sentences are lists of tokens.
    
    Parameters
    ----------
    encoding : string, 'utf-8' by default.
        If bytes or files are given to analyze, this encoding is used to
        decode.
    ngram_range : tuple (min_n, max_n)
        The lower and upper boundary of the range of n-values for different
        n-grams to be extracted. All values of n such that min_n <= n <= max_n
        will be used.
    max_df: float in range [0, 1] or int, default=1.0
    min_df: float in range [0, 1] or int, default=1
    Example
    -------
    >> import Cooccurrence
    >> docs = ['this book is good',
               'this cat is good',
               'cat is good shit']
    >> model = Cooccurrence()
    >> Xc = model.fit_transform(docs)
    Check vocabulary by printing
    >> model.vocabulary_
    """

    def __init__(self, encoding='utf-8', ngram_range=(1, 1),
                 max_df=1.0, min_df=1, max_features=None,
                 stop_words=None, normalize=False, vocabulary=None):

        super(Cooccurrence, self).__init__(
            ngram_range=ngram_range,
            max_df=max_df,
            min_df=min_df,
            max_features=max_features,
            stop_words=stop_words,
            vocabulary=vocabulary,
            analyzer=lambda x:x
        )
        self.normalize = normalize

    def fit_transform(self, raw_documents, y=None):
        """Fit cooccurrence matrix
        Parameters
        ----------
        raw_documents : iterable
            an iterable which yields either str, unicode or file objects
        Returns
        -------
        Xc : Cooccurrence matrix
        """
        X = super(Cooccurrence, self).fit_transform(raw_documents)
        n_samples, n_features = X.shape

        Xc = (X.T * X)
        if self.normalize:
            g = sp.diags(1./Xc.diagonal())
            Xc = g * Xc
        else:
            Xc.setdiag(0)

        return Xc

In [131]:
c = Cooccurrence()

In [132]:
sdocs = [s.split() for s in docs]; sdocs

[['this', 'this', 'this', 'book'],
 ['this', 'cat', 'good'],
 ['cat', 'good', 'shit']]

In [133]:
for t in sdocs[0]:
    print(f"{t}.")

this.
this.
this.
book.


In [134]:
cv = Cooccurrence()

In [135]:
xx = cv.fit_transform(sdocs)

In [136]:
cv.get_feature_names()

['book', 'cat', 'good', 'shit', 'this']

In [112]:
xx.todense()

matrix([[0, 0, 0, 0, 3],
        [0, 0, 2, 1, 1],
        [0, 2, 0, 1, 1],
        [0, 1, 1, 0, 0],
        [3, 1, 1, 0, 0]], dtype=int64)

In [114]:
cv.get_feature_names()

['book', 'cat', 'good', 'shit', 'this']

In [122]:
??CountVectorizer